In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import sklearn.linear_model as lm
import src.random_matrix as rm
import statsmodels.api as sm

PATH = '/data/hawfinch/clarkson/data/us_eq/cache'

In [20]:
df = pd.read_pickle('{}/log_returns_cleaned.pkl'.format(PATH))

In [53]:
def estimate_residuals(factors, returns):
    """
    :param factors: factors matrix of size kxn
    :param returns: stock returns of size nxp
    """
    fr = pd.DataFrame(np.matmul(factors.T, returns.values), columns=returns.columns.values)
    # Fit OLS to get betas
    return np.sum()

def estimate_betas(y, X):
    return lm.LinearRegression(intercept=False).fit(y, X).coef_

def compute_factor_return(factor, stock_returns, stdevs):
    return factor.dot(stock_returns.values / stdevs)

In [82]:
# df.rolling(60).apply(estimate_residual)
test_data = df.iloc[:60]
test_data.dropna(thresh=len(test_data.index)*.95, axis=1, inplace=True)
test_data.dropna(thresh=len(test_data.columns), axis=0, inplace=True)
test_data

/homes/clarkson/PycharmProjects/mini_1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/homes/clarkson/PycharmProjects/mini_1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


ticker,AA,AABA,AAON,AAPL,ABC,ABCB,ABM,ABMD,ABT,ACHC,...,XOM,XRAY,XRX,Y,YUM,ZBRA,ZEUS,ZION,ZIXI,PKG
date,,,,,,,,,,,,,,,,,,,,,
2000-02-01,0.037314,-0.014662,-0.068993,-0.034317,0.030563,0.012270,0.006079,-0.017242,0.009551,0.000000,...,0.003764,0.010050,-0.002999,0.000335,0.071609,0.006316,0.054067,0.010515,0.043571,-0.005195
2000-02-02,0.031142,0.032930,0.057158,-0.014443,0.009983,0.024098,-0.009132,-0.021979,0.005687,0.331357,...,0.007485,0.031983,0.005988,0.007346,-0.060752,-0.007902,0.000000,-0.019008,0.038027,0.005195
2000-02-03,-0.030032,0.093785,0.031253,0.044534,-0.006645,-0.024098,0.027151,-0.050124,0.005655,-0.108214,...,-0.017300,0.004831,0.038072,0.000333,0.025588,0.095263,0.000000,-0.011796,0.112795,-0.005195
2000-02-04,-0.025830,-0.018915,0.015268,0.044373,-0.016807,-0.012270,0.040822,0.029922,0.035091,-0.121361,...,-0.018377,0.014354,0.011429,0.000000,-0.002107,0.018104,0.000000,-0.046366,0.029559,-0.053489
2000-02-07,-0.032373,0.001413,-0.038615,0.054615,-0.006803,0.036368,-0.005731,0.070051,-0.018316,0.000000,...,-0.001547,0.007101,-0.014306,0.005968,-0.006349,0.015925,-0.026668,-0.001131,-0.009756,-0.050715
2000-02-08,-0.037108,0.052617,0.015625,0.007098,-0.024181,-0.024098,-0.026202,-0.040998,0.009200,-0.066691,...,-0.022702,-0.002361,0.002878,-0.000331,0.014753,0.012009,0.013423,0.049652,0.004890,-0.090697
2000-02-09,-0.018462,-0.029406,0.110001,-0.019781,-0.035591,0.000000,-0.002954,-0.035878,0.055223,0.066691,...,-0.034639,-0.011891,-0.005764,0.001652,0.004175,-0.009225,0.039221,-0.051917,-0.021365,0.025001
2000-02-10,0.014185,0.007390,0.030772,0.007739,-0.010929,-0.012270,0.020499,0.072639,-0.026340,0.077558,...,-0.004930,-0.019324,-0.041304,-0.007290,-0.087012,-0.001855,-0.052644,-0.003407,0.035893,0.048202
2000-02-11,0.031940,-0.063078,-0.003373,-0.042751,-0.033523,0.024391,-0.007273,0.002121,-0.032553,-0.045810,...,-0.026713,-0.009804,-0.024391,-0.002331,0.015784,-0.031120,-0.027399,-0.004561,-0.014528,-0.054394


In [86]:
pca = PCA(n_components=0.75, svd_solver='full')
pca.fit(test_data.values.T)
t = pca.transform(test_data.values.T)

print(t.T.shape)
print(test_data.values.shape)
# print(test_data.columns.values)
# fr = pd.DataFrame(np.matmul(t.T, test_data.values), columns=test_data.columns.values)
print(test_data.iloc[-1].values)
print(t.shape)
fr = np.dot(t.T, test_data.iloc[-1].values)
fr
# fr['AA'].name
# betas = fr.apply(lambda x: estimate_betas(test_data[x.name], )

# fr.shape
# print(estimate_betas(test_data['AA']))
# print(test_data.iloc[-1])

(31, 1024)
(58, 1024)
[-0.00740745 -0.07809874  0.01499232 ...  0.01156082 -0.06103589
  0.02325686]
(1024, 31)


array([-2.42640842,  0.17560631,  0.35852756, -0.907378  , -1.06613941,
       -0.13848263, -0.26557261,  0.32602506,  0.98488241, -0.03422161,
       -0.16495535, -0.44581772,  0.2244865 ,  0.19105742, -0.28753527,
        0.09963997,  0.35940131, -0.35301512, -0.28199037, -0.34405933,
       -0.4082466 , -0.13902731,  0.15161987, -0.28409229,  0.33010472,
        0.06396258,  0.22338559, -0.12506894,  0.13704726, -0.12980892,
        0.11561571])

In [44]:
x = np.ones(10)
print(x)
y = np.random.randint(10, size=10)
print(y)
x / y

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[7 9 8 3 8 7 9 4 4 0]


/homes/clarkson/PycharmProjects/mini_1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


array([0.14285714, 0.11111111, 0.125     , 0.33333333, 0.125     ,
       0.14285714, 0.11111111, 0.25      , 0.25      ,        inf])